In [1]:
import cv2

img = cv2.imread("D:/test_pic/2.jpg")
img = cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)

In [2]:
import numpy as np

# 放大的尺寸
new_width, new_height = (img.shape[1] + 10), (img.shape[0] + 5)

# 转换为灰度图像
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 计算能量值函数（梯度）
sobel_x = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=3)
energy_map = cv2.add(np.absolute(sobel_x), np.absolute(sobel_y))

# 选择需要删除的seam，直到达到目标尺寸
for i in range(img.shape[1] - new_width):
    # 计算每个通道的能量值
    energy_map_r = cv2.Sobel(img[:,:,2], cv2.CV_64F, 1, 0, ksize=3)
    energy_map_g = cv2.Sobel(img[:,:,1], cv2.CV_64F, 1, 0, ksize=3)
    energy_map_b = cv2.Sobel(img[:,:,0], cv2.CV_64F, 1, 0, ksize=3)
    energy_map = cv2.add(np.absolute(energy_map_r), np.absolute(energy_map_g))
    energy_map = cv2.add(energy_map, np.absolute(energy_map_b))

    # 找到能量值最小的seam
    min_energy_seam = np.zeros((img.shape[0]), dtype=np.int32)
    seam_energy = np.zeros((img.shape[0], img.shape[1]))
    for j in range(img.shape[0]):
        if j == 0:
            seam_energy[j,:] = energy_map[j,:]
        else:
            for k in range(img.shape[1]):
                if k == 0:
                    seam_energy[j][k] = energy_map[j][k] + min(seam_energy[j-1][k], seam_energy[j-1][k+1])
                elif k == img.shape[1]-1:
                    seam_energy[j][k] = energy_map[j][k] + min(seam_energy[j-1][k-1], seam_energy[j-1][k])
                else:
                    seam_energy[j][k] = energy_map[j][k] + min(seam_energy[j-1][k-1], seam_energy[j-1][k], seam_energy[j-1][k+1])
        min_energy_seam[j] = np.argmin(seam_energy[-1,:])

    # 删除能量值最小的seam
    for j in range(img.shape[0]):
        img[j, min_energy_seam[j]:-1, :] = img[j, min_energy_seam[j]+1:, :]
    
# 缩小图像以达到目标尺寸
img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

# 显示图像
cv2.imshow('Resized Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("D:/test_pic/fd_2.jpg",img)

True